In [23]:
import pickle
import numpy
import os
import wget
from pydub import AudioSegment
import librosa
import numpy as np
import math

In [4]:
model_age = pickle.load(open('saved_model/age_xgb.sav', 'rb'))
model_sex = pickle.load(open('saved_model/gender_xgb.sav', 'rb'))

In [6]:
directory = 'saved_model/single_pred'

if not os.path.exists(directory):
    os.makedirs(directory)

In [14]:
input_size = 13*431
def audio_to_mfcc(audio_file):
    
    wav,sr = librosa.load(audio_file)
    if wav.shape[0]<10*sr:
        wav=np.pad(wav,int(np.ceil((10*sr-wav.shape[0])/2)),mode='reflect')
    else:
        wav=wav[:10*sr]
        
    mfcc = librosa.feature.mfcc(wav, n_mfcc=13, sr=sr)
    return mfcc.reshape(-1, input_size)

In [26]:
def predict(url):
    url = url.replace('https://voca.ro/', 'https://media.vocaroo.com/mp3/')
    file_id = url.replace('https://media.vocaroo.com/mp3/', 'Vocaroo ') + '.mp3'
    
    try:
        file = wget.download(url, out=directory)
    except: 
        print("Audio Does not exist")
        
    source = '{}/{}'.format(directory, file_id)   
    destination = source.replace('.mp3', '.wav')
    
    sound = AudioSegment.from_file(source)
    sound.export(destination, format="wav")

    audio = audio_to_mfcc(destination)
    
    age_pred = model_age.predict(audio)
    sex_pred = model_sex.predict(audio)
    
    print()
    print()
    
    print("Predicted Age: ", math.floor(age_pred[0]))
    if sex_pred[0] == 0:
        print("Predicted Gender: Male")
    else:
        print("Predicted Gender: Female")
    
    
    
    pass

# Single Prediction

Please Visit this website to record your audio. 

https://vocaroo.com/

Save and share the link in the input field. 

#### Note: The Audio must be 10 seconds or more in length

In [31]:
url = input()
predict(url)

https://voca.ro/1a4lbydDUIdL
100% [............................................................................] 134496 / 134496

Predicted Age:  46
Predicted Gender: Male
